In [ ]:
# required libraries
import os
import pandas as pd
import numpy as np
from pathlib import Path
# # Load the required libraries 
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# # rdflib knows about some namespaces, like FOAF 
from rdflib.namespace import FOAF, XSD
# CHECK DATE 
import datetime

In [ ]:
# parameters and URLs"
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())
print("Here is the path "+ path)
path = 'C:/Users/tamim/Desktop/semproject/csv'
appearancesUrl = path + '/appearances.csv'
clubGamesUrl = path + '/club_games.csv'
clubUrl = path + '/clubs.csv'
competitionsUrl = path + '/competitions.csv'
gameEventsUrl = path + '/game_events.csv'
gameLineupsUrl = path + '/game_lineups.csv'
gamesUrl = path + '/games.csv'
playerValuationsUrl = path + '/player_valuations.csv'
playersUrl = path + '/players.csv'
# country code
countriesURL = path + '/wikipedia-iso-country-codes.csv'

print("File paths are working perfectly.")
# saving folder
savePath =  path + '/data/'
print("executed all lines")

In [ ]:
# Load the CSV files in memory

print("load csv files")
comp = pd.read_csv(competitionsUrl, sep = ',', index_col = 'competition_code',keep_default_na=False, na_values=['_'])
comp['domestic_league_code'] = comp['domestic_league_code'].fillna('')
print("there is a problem")
comp.astype({'country_id': 'int32'}).dtypes

In [ ]:
#load the country codes\
# we need to convert NaN values to something else otherwise NA strings are converted to NaN -> problem with Namibia
countries = pd.read_csv(countriesURL, sep=',', index_col='Name', keep_default_na=False, na_values=['_'])

In [ ]:
comp.info()

In [ ]:
# Construct the country and the football ontology namespaces not known by RDFlib\n",
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
FO = Namespace("http://www.dei.unipd.it/database2/FootballOntology#")


In [ ]:
#create the graph\
g = Graph()
# Bind the namespaces to a prefix for more readable output\n",
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)


In [ ]:
%%time

for index, row in comp.iterrows():
    #Create the node to add to the Graph
    #the node has the namespace + the movie id as URI
    Competition = URIRef(FO[index])
    # Add triples using store's add() method.
    g.add((Competition, RDF.type, FO.Competition))
    g.add((Competition, FO['competitionID'], Literal(row['competition_id'], datatype=XSD.string)))
    g.add((Competition, FOAF['name'], Literal(row['name'], datatype=XSD.string)))
    g.add((Competition, FO['competitionType'], Literal(row['type'], datatype=XSD.string)))
    g.add((Competition, FO['subType'], Literal(row['sub_type'], datatype=XSD.string)))
    g.add((Competition, FO['domesticLeagueCode'], Literal(row['domestic_league_code'], datatype=XSD.string)))                                                
    g.add((Competition, FO['country_id'], Literal(row['country_id'], datatype=XSD.integer)))
    g.add((Competition, FO['confederation'], Literal(row['confederation'], datatype=XSD.string)))
    ## handle country
    #there can be more than one country per competition
    for c in str(row['country']).split(','):
        # check if the country exists
        # country.index == x returns an array of booleans, thus we need to use the any() method
        cName = c.strip()
        if((countries.index == cName).any() == True):
            #get the country code, convert to string and get the lower case to match the country codes in the ontology
            code = str(countries[countries.index == cName]['Alpha-2 code'][0]).lower()
            # create the RDF node
            Country = URIRef(CNS[code])
             # add the edge connecting the Competition and the Country
            g.add((Competition, FO['hasCountry'], Country))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'competitions.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
# Load the CSV files in memory

print("load csv files")
club = pd.read_csv(clubUrl, sep = ',',index_col='club_code', keep_default_na=False, na_values=['_'])
club['domestic_competition_id'] = club['domestic_competition_id'].fillna('')
club.info()


In [ ]:
# Load the CSV files in memory\n",


club = pd.read_csv(clubUrl, sep=',', index_col='club_code', keep_default_na=False, na_values=['_'])
club.astype({'club_id': 'string'}).dtypes
club['average_age'] = club['average_age'].replace('', '0.0')
club['average_age'] = club['average_age'].astype('float64')
club['foreigners_percentage'] = club['average_age'].replace('', '0.0')
club['foreigners_percentage'] = club['average_age'].astype('float64')


In [ ]:
club.info()

In [ ]:
%%time


for index, row in club.iterrows():
    Clubs = URIRef(FO[index]) 
    # Add triples for each row
    g.add((Clubs, RDF.type, FO.Club))
    g.add((Clubs, FO['clubID'], Literal(row['club_id'], datatype=XSD.string)))
    g.add((Clubs, FOAF['name'], Literal(row['name'], datatype=XSD.string)))
    g.add((Clubs, FO['domesticCompetitionID'], Literal(row['domestic_competition_id'], datatype=XSD.string)))
    g.add((Clubs, FO['averageAge'], Literal(row['average_age'], datatype=XSD.decimal)))
    g.add((Clubs, FO['lastSeason'], Literal(row['last_season'], datatype=XSD.gYear)))
    g.add((Clubs, FO['squadSize'], Literal(row['squad_size'], datatype=XSD.integer)))
    g.add((Clubs, FO['foreigners'], Literal(row['foreigners_number'], datatype=XSD.integer)))
    g.add((Clubs, FO['foreignersPercentage'], Literal(row['foreigners_percentage'], datatype=XSD.decimal)))
    g.add((Clubs, FO['nationalTeamPlayers'], Literal(row['national_team_players'], datatype=XSD.integer)))
    

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'club.ttl', 'w',encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
# Load the CSV files in memory\n",
print("load csv files")
players = pd.read_csv(playersUrl, sep = ',', index_col = 'player_code')
players.info()
players.astype({'player_id': 'string'}).dtypes
players.astype({'current_club_id': 'string'}).dtypes
players['agent_name'] = players['agent_name'].fillna('')
players['height_in_cm'] = players['height_in_cm'].fillna(players['height_in_cm'].mean())
players['market_value_in_eur'] = players['market_value_in_eur'].fillna(round(players['market_value_in_eur'].mean(),2))

In [ ]:
players.info()

In [ ]:
%%time

for index, row in players.iterrows():
    #Create the node to add to the Graph
    #the node has the namespace + the movie id as URI
    player = URIRef(FO[index])
    # Add triples using store's add() method.
    g.add((player, RDF.type, FO.Player))
    g.add((player, FO['playerID'], Literal(row['player_id'], datatype=XSD.string)))
    g.add((player, FOAF['firstName'], Literal(row['first_name'], datatype=XSD.string)))
    g.add((player, FOAF['lastName'], Literal(row['last_name'], datatype=XSD.string)))
    g.add((player, FOAF['name'], Literal(row['name'], datatype=XSD.string)))
    g.add((player, FO['lastSeason'], Literal(row['last_season'], datatype=XSD.gYear)))                                                
    g.add((player, FO['DomesticCompetition'], Literal(row['current_club_domestic_competition_id'], datatype=XSD.string)))
    g.add((player, FO['ClubID'], Literal(row['current_club_id'], datatype=XSD.string)))
    g.add((player, FO['ClubName'], Literal(row['current_club_name'], datatype=XSD.string)))
    g.add((player, FO['height'], Literal(row['height_in_cm'], datatype=XSD.integer)))
    g.add((player, FO['cityOfBirth'], Literal(row['city_of_birth'], datatype=XSD.string)))
    t = row['country_of_birth']
    if((countries.index == t).any() == True):
        code = str(countries[countries.index == t]['Alpha-2 code'][0]).lower()
        bCountry = URIRef(CNS[code])
    g.add((player, FO['birthCountry'], bCountry))
    ## handle country
    #there can be more than one country per competition
    for c in str(row['country_of_citizenship']).split(','):
        # check if the country exists
        # country.index == x returns an array of booleans, thus we need to use the any() method
        cName = c.strip()
        if((countries.index == cName).any() == True):
            #get the country code, convert to string and get the lower case to match the country codes in the ontology
            code = str(countries[countries.index == cName]['Alpha-2 code'][0]).lower()
            # create the RDF node
            Country = URIRef(CNS[code])
             # add the edge connecting the Competition and the Country
            g.add((player, FO['hasCountry'], Country))
    g.add((player,FO['birthDate'], Literal(row['date_of_birth'], datatype=XSD.date)))
    g.add((player, FO['position'], Literal(row['position'], datatype=XSD.string)))
    g.add((player, FO['agentName'], Literal(row['agent_name'], datatype=XSD.string)))
    g.add((player, FO['marketValue'], Literal(row['market_value_in_eur'], datatype=XSD.decimal)))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'player.ttl', 'w', newline='', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
# Load the CSV files in memory

print("load csv files")
games = pd.read_csv(gamesUrl, sep = ',', index_col = 'url')
print("there is a problem")
games.info()

In [ ]:
%%time

for index, row in  games.iterrows():
    #Create the node to add to the Graph
    #the node has the namespace + the movie id as URI
    Game = URIRef(FO[index])
    # Add triples using store's add() method.
    g.add((Game, RDF.type, FO.Game))
    g.add((Game, FO['gameID'], Literal(row['game_id'], datatype=XSD.integer)))
    g.add((Game, FO['competitionID'], Literal(row['competition_id'], datatype=XSD.string)))
    g.add((Game, FO['season'], Literal(row['season'], datatype=XSD.gYear)))
    g.add((Game, FO['date'], Literal(row['date'], datatype=XSD.date)))
    g.add((Game, FO['homeClubID'], Literal(row['home_club_id'], datatype=XSD.string)))                                            
    g.add((Game, FO['awayClubID'], Literal(row['away_club_id'], datatype=XSD.string)))
    g.add((Game, FO['homeClubGoals'], Literal(row['home_club_goals'], datatype=XSD.integer)))
    g.add((Game, FO['awayClubGoals'], Literal(row['away_club_goals'], datatype=XSD.integer)))
    g.add((Game, FO['referee'], Literal(row['referee'], datatype=XSD.string)))
  

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'games.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
# Load the CSV files in memory\n",
player_val = pd.read_csv(playerValuationsUrl, sep=',', index_col='valuation_id', keep_default_na=False, na_values=['_'])

In [ ]:
# Load the CSV files in memory

print("load csv files")
player_val = pd.read_csv(playerValuationsUrl, sep = ',', index_col = 'valuation_id')
print("there is a problem")

player_val['date'] = pd.to_datetime(player_val['date'])
player_val['date']  = player_val['date'].dt.strftime('%Y-%m-%d')
player_val['market_value_in_eur'] =  pd.to_numeric(player_val['market_value_in_eur'], errors='coerce').astype(float)
player_val['market_value_in_eur'] = player_val['market_value_in_eur'].fillna(player_val['market_value_in_eur'].mean())



In [ ]:
%%time

for index, row in player_val.iterrows():
    #Create the node to add to the Graph
    #the node has the namespace + the movie id as URI
    Valuation = URIRef(FO[index])
    # Add triples using store's add() method.
    g.add((Valuation, RDF.type, FO.Valuation))
    g.add((Valuation, FO['playerID'], Literal(row['player_id'], datatype=XSD.string)))
    g.add((Valuation, FO['date'], Literal(row['date'], datatype=XSD.date)))
    g.add((Valuation, FO['marketValue'], Literal(row['market_value_in_eur'], datatype=XSD.decimal)))                                                
    g.add((Valuation, FO['clubID'], Literal(row['current_club_id'], datatype=XSD.string)))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'valuation.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
print("load csv files")
gameEvents = pd.read_csv(gameEventsUrl, sep = ',', index_col = 'game_event_id', keep_default_na=False, na_values=['_'])
gameEvents['description']=gameEvents['description'].fillna('')
gameEvents['type']=gameEvents['type'].fillna('')
gameEvents['minute']=gameEvents['minute'].fillna(gameEvents['minute'].mean())


In [ ]:
gameEvents.info()

In [ ]:
%%time

for index, row in gameEvents.iterrows():
    #Create the node to add to the Graph
    #the node has the namespace + the player id as URI
    GameEvent = URIRef(FO[index])
    # Add triples using store's add() method.
    g.add((GameEvent, RDF.type, FO.GameEvent))
    g.add((GameEvent, FO['date'], Literal(row['date'], datatype=XSD.date)))
    g.add((GameEvent, FO['gameID'], Literal(row['game_id'], datatype=XSD.string)))
    g.add((GameEvent, FO['minute'], Literal(row['minute'], datatype=XSD.integer)))
    g.add((GameEvent, FO['eventType'], Literal(row['type'], datatype=XSD.string)))
    g.add((GameEvent, FO['clubID'], Literal(row['club_id'], datatype=XSD.string)))                                                                                    
    g.add((GameEvent, FO['playerID'], Literal(row['player_id'], datatype=XSD.string)))
    g.add((GameEvent, FO['description'], Literal(row['description'], datatype=XSD.string)))
    g.add((GameEvent, FO['playerInID'], Literal(row['player_in_id'], datatype=XSD.string)))
    g.add((GameEvent, FO['playerAssistID'], Literal(row['player_assist_id'], datatype=XSD.string)))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'GameEvents.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
print("load csv files")
gameLineUp = pd.read_csv(gameLineupsUrl, sep = ',', index_col = 'game_lineups_id')
print("there is a problem")
gameLineUp.astype({'team_captain': 'int32'}).dtypes


In [ ]:
gameLineUp.info()

In [ ]:
%%time

for index, row in gameLineUp.iterrows():
    #Create the node to add to the Graph
    #the node has the namespace + the player id as URI
    GameLineUp = URIRef(FO[index])
    # Add triples using store's add() method.
    g.add((GameLineUp, RDF.type, FO.GameLineUp))
    
    g.add((GameLineUp, FO['ClubID'], Literal(row['club_id'], datatype=XSD.string)))
    g.add((GameLineUp, FO['lineUpType'], Literal(row['type'], datatype=XSD.string)))
    g.add((GameLineUp, FO['PlayerID'], Literal(row['player_id'], datatype=XSD.string)))
    g.add((GameLineUp, FO['PlayerName'], Literal(row['player_name'], datatype=XSD.string)))                                           
    g.add((GameLineUp, FO['PlayerPosition'], Literal(row['position'], datatype=XSD.string)))                                                                                                
    g.add((GameLineUp, FO['gameID'], Literal(row['game_id'], datatype=XSD.string)))
    g.add((GameLineUp, FO['jerseyNumber'], Literal(row['number'], datatype=XSD.string)))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'GameLineUp.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
print("load csv files")
appearances = pd.read_csv(appearancesUrl, sep = ',', index_col = 'appearance_id')
appearances.info()
appearances.astype({'player_id': 'string'}).dtypes
appearances.astype({'player_club_id': 'string'}).dtypes
appearances.astype({'player_current_club_id': 'string'}).dtypes
# Convert the column to date format
appearances['date'] = pd.to_datetime(appearances['date'])
appearances['date']  = appearances['date'].dt.strftime('%Y-%m-%d')

In [ ]:
%%time

for index, row in appearances.iterrows():
    #Create the node to add to the Graph
    #the node has the namespace + the appearance id as URI
    Appearance = URIRef(FO[index])
    # Add triples using store's add() method.
    g.add((Appearance, RDF.type, FO.Appearance))
    
    g.add((Appearance, FO['gameID'], Literal(row['game_id'], datatype=XSD.string)))
    g.add((Appearance, FO['playerID'], Literal(row['player_id'], datatype=XSD.string)))
    g.add((Appearance, FO['clubID'], Literal(row['player_club_id'], datatype=XSD.string)))
    g.add((Appearance, FO['date'], Literal(row['date'], datatype=XSD.date)))                                           
    g.add((Appearance, FO['playerName'], Literal(row['player_name'], datatype=XSD.string)))                                                                                                
    g.add((Appearance, FO['yellowCards'], Literal(row['yellow_cards'], datatype=XSD.integer)))
    g.add((Appearance, FO['redCards'], Literal(row['red_cards'], datatype=XSD.integer)))
    g.add((Appearance, FO['assists'], Literal(row['assists'], datatype=XSD.integer)))
    g.add((Appearance, FO['minutesPlayed'], Literal(row['minutes_played'], datatype=XSD.integer)))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'Appearances.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))